# FUNCTION EVALUATION FILE

In [263]:
from prepare import *
ecs=equalize_and_compress_and_sort

In [264]:
function=  """
  functions   //A fuction which returns a number between 0 and 1 given a number than might step outside of the bounds
    {
     real makep(real p)
        {real q;
           {if (p<=0)
               q=10^-16;
            else
               {if (p>=1)
                  q=1;
                else
                  q=p;
               }
            }
         return q;  
        }
    
    real uf(real x,real alpha,real beta,real lamda) //the utility functiton
        { real z;
            {if (x>=0)
                z=(fabs(x)^alpha);
             else
                z=-lamda*(fabs(x)^beta);
            }
        return z;    
        }
        
    real iuf(real x,real alpha,real beta,real lamda) //the inverse utility function
        { real z;
            {if (x>=0)
                z=fabs(x)^(1/alpha);
             else
                z=-((fabs(x)/lamda)^(1/beta));
            }
        return z;    
        }    
        
    real pf(real p,real alpha,real beta)             // probability weighting function
        {real z; real h;
               { h=makep(p);
                 z=exp(-beta*(((-log(h))^alpha)));
            } 
        return z;    
        }  
    vector vpf(vector p,int N,real alpha,real beta)  //vector probability weighting funciton
        {
         vector[N] z; 
         for(i in 1:N) 
            {
            z[i]=pf(p[i],alpha,beta);
            } 
        return z;
        }    
    vector vuf(vector x,int N,real alpha,real beta,real lamda)  //vector utility function
        {
        vector[N] z; 
        for(i in 1:N) 
            {
            z[i]=uf(x[i],alpha,beta,lamda);
            } 
        return z;
        }
    real evf(vector x,int N,vector w, vector W)             //expected value function
        {
        real z;
        z=0;
        for(i in 1:N) 
            {if (x[i]>=0)
                z=z+w[i]*x[i];
             else
                z=z+W[i]*x[i];
            } 
        return z;
        }  
     vector adcf(vector p,int N)                              //anti-decumulative function
        {
        vector[N] z; int i;
        for(j in 1:N) 
            {i=N-j+1;
            if (i==N)
                z[i]=p[i];
            else
                z[i]=p[i]-p[i+1];
            } 
        return z;
        }
      vector acf(vector p,int N)                               //anti-cumulative function
        {
        vector[N] z; 
        for(i in 1:N) 
            {
            if (i==1)
                z[i]=p[i];
            else
                z[i]=p[i]-p[i-1];
            } 
        return z;
        }  
    }

data {int typ;              //positive if utility model else uses certainty equivalents
       int N;               //Number of payoffs in each lottery
       int M;               //Number of Choices made by each individual (must be same for all)
       int T;               //Number of people 
       int y[T,M];          //The choices, 1 for option A, 2 for B 
       
       vector[N] xA[T,M];   //Payoffs for option A
       vector[N] xB[T,M];   //Payoffs for option B
       vector[N] FA[T,M];   //Decumulative Distributions for option A
       vector[N] PA[T,M];   //Cumulative Distributions for option A
       vector[N] FB[T,M];   //Decumulative Distributions for option B
       vector[N] PB[T,M];   //Cumulative Distributions for option B
       
       
       //The following are the prior means for the mean parameters
       //For alpha,beta,lambda, gammas and deltas these can be =1, m_scale=1 is also ok
       real m_alpha;                         //Power for value function in the gain doman
       real m_beta;                          //Power for value function in the loss doman
       real m_lamda;                         // Loss Aversion Coefficient
       real m_gama1;   real m_gama2;         //probability weightings Gain Domain (for Prelec II)
       real m_delta1;  real m_delta2;        //probability weightings Gain Domain (for Prelec II)
       }

parameters{//The mean parameters, those these are not actually used here
           real alpha;  //Power Utility in the Gain Domain
           real beta;   //Power Utility in the Loss Domain
           real lamda;  //Loss Aversion Parameter
           
           real gama1;    //Prelec II probability Weightings Gain Domain
           real gama2; 
           
           real delta1;   //Prelec II probability Weightings Loss Domain
           real delta2;
       }
           
//The transformed parameters are below
//While these are declared directly below, their definitions follow within the transformed parameters block

transformed parameters {vector[N] vA[T,M]; vector[N] vB[T,M];
                        vector[N] FA_[T,M];vector[N] PA_[T,M]; 
                        vector[N] FB_[T,M];vector[N] PB_[T,M];  
                        vector[N] WFA[T,M]; vector[N] WPA[T,M];
                        vector[N] WFB[T,M]; vector[N] WPB[T,M];
                        
                        real evA_[T,M]; 
                        real evB_[T,M]; 
                        real euA_[T,M]; 
                        real euB_[T,M]; 
                        real CEA_[T,M];  
                        real CEB_[T,M];
                        
                        vector[2] MU[T,M];
                        
                        
                       
                        for (t in 1:T)
                        {
                        for (i in  1:M){
                        vA[t,i]=vuf(xA[t,i],N,m_alpha,m_beta,m_lamda);  //vector of utilities option A
                        vB[t,i]=vuf(xB[t,i],N,m_alpha,m_beta,m_lamda);  //vector of utilities option B
                        
                        PA_[t,i]=vpf(PA[t,i],N,m_delta1,m_delta2);        //cumulative transformed A
                        FA_[t,i]=vpf(FA[t,i],N,m_gama1,m_gama2);          //decumulative transformed A
                        
                        PB_[t,i]=vpf(PB[t,i],N,m_delta1,m_delta2);         //cumulative transformed B
                        FB_[t,i]=vpf(FB[t,i],N,m_gama1,m_gama2);           //decumulative transformed B
                               
                        WPA[t,i]=acf(PA_[t,i],N);                              //weights loss domain A
                        WFA[t,i]=adcf(FA_[t,i],N);                             //weights gain domain A
                        
                        WPB[t,i]=acf(PB_[t,i],N);                              //weights loss domain B
                        WFB[t,i]=adcf(FB_[t,i],N);                             //weights gain domain B
                        
                        euA_[t,i]=evf(vA[t,i],N,WFA[t,i],WPA[t,i]);           // PT expected utility A
                        euB_[t,i]=evf(vB[t,i],N,WFB[t,i],WPB[t,i]);            // PT expected utility B
                        
                                               
                        evA_[t,i]=evf(xA[t,i],N,WFA[t,i],WPA[t,i]);             // Expected Value of A based on PT weights
                        evB_[t,i]=evf(xB[t,i],N,WFB[t,i],WPB[t,i]);             // Expected Value of B based on PT weights
                        
                      
                        CEA_[t,i]=iuf(euA_[t,i],m_alpha,m_beta,m_lamda);     // CE under PT
                        CEB_[t,i]=iuf(euB_[t,i],m_alpha,m_beta,m_lamda);     // CE under PT 
 
                        
                                                                  //Do Utility model
                        MU[t][i,1]=euA_[t,i]; 
                        MU[t][i,2]=euB_[t,i];              
                        
                                      }         
                        
                        }
                        }
                                               
model {
     //The priors for the parameters are set out below
       alpha~normal(m_alpha,1);     
       beta~normal(m_beta,1);        
       lamda~normal(m_lamda,1);     
       gama1~normal(m_gama1,1);     
       gama2~normal(m_gama2,1);     
       delta1~normal(m_delta1,1);  
       delta2~normal(m_delta2,1); 
       
       for(t in 1:T)
              for(i in 1:M) {  
                                y[t,i] ~ categorical_logit(MU[t][i]);
                             }
              
              
              
      }
      

"""

In [265]:
sm = pystan.StanModel(model_code=function)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5ac9a57a2986b58ca9ce65adb5c089b0 NOW.


In [266]:
#The data contains 14300 rows and 45 columns, it has different treatments etc but we will not use these
dat=pd.read_excel('Fulldata3.xlsx')

In [267]:
#Create a variable specifying the domain of the gamble as 1,2,3 below
dat['Dom']=(dat['Gains']==1)*1+(dat['Loss']==1)*2+(dat['Mixed']==1)*3

#All lotteries have up to three payoffs. However, there are missing values that should be given zeros
xA=twodma(dat[['prize1l','prize2l','prize3l']].fillna(0))
pA=twodma(dat[['p1left','p2left','p3left']].fillna(0))
pA=pA/sumc(pA.T)
xB=twodma(dat[['prize1r','prize2r','prize3r']].fillna(0))
pB=twodma(dat[['p1right','p2right','p3right']].fillna(0))
pB=pB/sumc(pB.T)

#The lotteries are not ordered and can contain repeated values, 
#The ecs function (written in python) compressess them into unique value and orders 
xA_,pA_=ecs(xA,pA)
xB_,pB_=ecs(xB,pB)

#Create the cumulative and decumulative vectors for options A and B
PA=cumdist(pA_)
FA=dcumdist(pA_)
PB=cumdist(pB_)
FB=dcumdist(pB_)

Y=array(-dat['choices']+2) #relabel the choices 1 for A and 2 for B, originally 0 and 1
DOM=array(dat['Dom'])
ideas=sorted(list(set(dat['id'])))

In [268]:

#Now create the arrays that can be read in a format that can be put into STAN
#The dimensions will match that in the Stan Code
y,xa,xb,fa,fb,pa,pb,dom=[],[],[],[],[],[],[],[]


M=100  #The number of tasks performed by each person

#Data is organised into responses by individuals
#Since the lotteries are repeated we will only calculate the utilites for the first individual
j=0
for i in range(1):
    y+=[Y[j:j+M]]      #The Choices
    xa+=[xA_[j:j+M,:]] #Lottery A Payoffs
    xb+=[xB_[j:j+M,:]] #Lottery B Payoffs
    fa+=[FA[j:j+M,:]]  #DeCumulative distribution for A
    fb+=[FB[j:j+M,:]]  #DeCumulative distribution for B
    pa+=[PA[j:j+M,:]]  #Cumulative distribution for A 
    pb+=[PB[j:j+M,:]]  #Cumulative distribution for B
    dom+=[DOM[j:j+M]]  #Specify the domain of the lottery
    j=j+M
    #print(j,j+M,rows(xA_[j:j+M,:]))
    
N=3           #The potential number of payoffs
T=1  #The number of people

In [269]:
#You can see the dimensions here
[shape(y),shape(xa),shape(xb),shape(fa),shape(fb),shape(pa),shape(pb),shape(dom)]

[(1, 100),
 (1, 100, 3),
 (1, 100, 3),
 (1, 100, 3),
 (1, 100, 3),
 (1, 100, 3),
 (1, 100, 3),
 (1, 100)]

In [270]:
sd=0.33  #The standard deviation for the prior mean of the key parameters

data={'typ': 1,  #Negative number if CE model otherwise utility
      'T':T,     #Number of people
      'N':N,      #Number of payoffs in each lottery
      'M':M,      #Number of choices made by an individual
      'y':y,         #Choices, 1 or 2, 
      'Domain':dom,  #Gains 1, Loss 2, Mixed 3.
      
      'xA':xa, # T by M array of N vectors 
      'xB':xb, # T by M array of N vectors 
      
      'FA':fa, # T by M array of N vectors    #Decumulative distribution of Lottery A 
      'PA':pa, # T by M array of N vectors    #Cumulative distribution of lottery A
    
      'FB':fb, # T by M array of N vectors    #Decumulative distribution of Lottery A
      'PB':pb, # T by M array of N vectors    #Decumulative distribution of Lotter B
      
      #These values are the values for which the utilities will be calculated below
      'm_alpha':.5,'m_beta':.6,'m_lamda':2,     #Prior neans of utility parameters
      'm_gama1':.6,'m_gama2':.5,                #Prior Means of prob weighting parameters in gain domain  
      'm_delta1':.4,'m_delta2':1.5,             #Prior means of prob weighting parameters in loss domain

   
     }

In [271]:
pars=('alpha','MU')
fit = sm.sampling(data=data,iter=3,warmup=2, chains=1,thin=1,pars=pars)

In [272]:
z=pull(fit)

alpha (1,)
MU (1, 1, 100, 2)
lp__ (1,)


The Marginal Utilities for the first 20 lotteries are

In [273]:
MU=z['MU']
MU=frame(squeeze(MU))
MU.iloc[0:20]

,0,1
0,-5.137535,-10.155113
1,-5.699654,-9.743317
2,-7.155204,-8.882572
3,-2.094388,-7.962143
4,-5.911178,-7.474386
5,-5.483178,-6.964405
6,-1.179560,-5.860312
7,-0.003672,-3.866364
8,0.973570,-3.031433
9,1.379156,0.000000
